In [12]:
import os
from pathlib import Path
from torchvision import transforms
import torch

In [13]:
from utils import dataloader, engine, save_load
from models import model_builder

In [14]:
base_dir = Path("data/")
train_path = base_dir / 'dataset_1' / 'train'
test_path = base_dir / 'dataset_1'/ 'test'

In [15]:
print("Train path:", train_path)
print("Exists?", os.path.exists(train_path))
print("Contains:", os.listdir(train_path))

Train path: data\dataset_1\train
Exists? True
Contains: ['french_toast', 'garlic_bread']


In [16]:
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [18]:
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

In [19]:
train_dataloader, test_dataloader, class_names = dataloader.create_dataloaders(
    test_dir=test_path,
    train_dir=train_path,
    transform=data_transform,
    batch_size=BATCH_SIZE,
)

In [20]:
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

In [21]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

In [ ]:
history = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
    device=device,
)

Training Epochs:  20%|██        | 1/5 [00:10<00:43, 10.88s/it]

Epoch: 1/5 | Train Loss: 0.6978 | Train Acc: 0.4489 | Test Loss: 0.6960 | Test Acc: 0.4479 |


Training Epochs:  40%|████      | 2/5 [00:20<00:29,  9.91s/it]

Epoch: 2/5 | Train Loss: 0.6926 | Train Acc: 0.5000 | Test Loss: 0.6920 | Test Acc: 0.4479 |


Training Epochs:  60%|██████    | 3/5 [00:29<00:19,  9.61s/it]

...

Training Epochs:  80%|████████  | 4/5 [00:38<00:09,  9.55s/it]

Epoch: 4/5 | Train Loss: 0.6902 | Train Acc: 0.5909 | Test Loss: 0.6911 | Test Acc: 0.4479 |


Training Epochs: 100%|██████████| 5/5 [00:48<00:00,  9.73s/it]

Epoch: 5/5 | Train Loss: 0.6931 | Train Acc: 0.4574 | Test Loss: 0.6834 | Test Acc: 0.5597 |


{'train_loss': [0.6977801377123053,
  0.6926366036588495,
  0.6933344548398798,
  0.6901964870366183,
  0.693079336123033],
 'train_acc': [0.44886363636363635,
  0.5,
  0.5397727272727273,
  0.5909090909090909,
  0.45738636363636365],
 'test_loss': [0.696006178855896,
  0.6920322775840759,
  0.6903409163157145,
  0.691084603468577,
  0.6833632191022238],
 'test_acc': [0.4479166666666667,
  0.4479166666666667,
  0.6354166666666666,
  0.4479166666666667,
  0.5596590909090909]}

In [23]:
save_load.save_model(model=model,
                 target_dir="models",
                 model_name="tiny_vgg_model_1.pth")

[INFO] Saving model to: models\tiny_vgg_model_1.pth
